<a href="https://colab.research.google.com/github/09eesx/FeelTrack-GenaritveAIFinalExam/blob/main/model_train_robertaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers==4.45.2 sentence-transformers==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [ ]:
!pip install transformers datasets scikit-learn --quiet


In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, DataCollatorWithPadding
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn as nn
from transformers import AdamW, get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
""""
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, DataCollatorWithPadding
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn as nn
from transformers import AdamW, get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")"""


# 1. Veriyi yükle ve etiketleri dönüştür
df = pd.read_csv("/content/drive/MyDrive/data/train_cleaned_augamented_cleaned.csv")
le = LabelEncoder()
df["label_id"] = le.fit_transform(df["label"])

# Oversampling uygula
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(df[["text"]], df["label_id"])
resampled_df = pd.DataFrame({"text": X_resampled["text"], "label_id": y_resampled})


# 2. Train / Val ayır
train_df, val_df = train_test_split(resampled_df, test_size=0.2, stratify=resampled_df["label_id"], random_state=42)

# 6. Class Weights hesapla
class_weights = compute_class_weight(class_weight="balanced",
                                     classes=np.unique(train_df["label_id"]),
                                     y=train_df["label_id"])
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

# 🔹 3. HuggingFace Dataset formatı
train_dataset = Dataset.from_pandas(train_df[["text", "label_id"]])
val_dataset = Dataset.from_pandas(val_df[["text", "label_id"]])

#sınıf sayısı değişebileceği için num_nabels
num_labels = df["label_id"].nunique()
# 🔹 4. Tokenizer ve Model
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion",
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

# 🔧 5. Tokenization fonksiyonu
def tokenize(example):
    encoding = tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
    encoding["labels"] = int(example["label_id"])
    return encoding

tokenized_train = train_dataset.map(tokenize)
tokenized_val = val_dataset.map(tokenize)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

"""model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion",
    num_labels=df["label_id"].nunique(),
    ignore_mismatched_sizes=True
).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

total_steps = len(train_dataloader) * 4  # 4 epoch
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps
)


class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # ✅ **kwargs eklendi
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

loss_fn = nn.CrossEntropyLoss()

for epoch in range(4):
    model.train()
    total_loss = 0
    loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch.pop("label_id")

        outputs = model(**batch)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"📉 Epoch {epoch+1} Training Loss: {total_loss / len(train_dataloader):.4f}")

    # 🔍 Validation
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch.pop("label_id")
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f"✅ Validation Accuracy: {acc:.4f}")



from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
save_path = f"/content/drive/MyDrive/data/roberta_model_{timestamp}"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model başarıyla kaydedildi: {save_path}")"""


In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-emotion",
    num_labels=df["label_id"].nunique(),
    ignore_mismatched_sizes=True
).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

total_steps = len(train_dataloader) * 4  # 4 epoch
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps
)


class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # ✅ **kwargs eklendi
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

loss_fn = nn.CrossEntropyLoss()

for epoch in range(4):
    model.train()
    total_loss = 0
    loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch.pop("label_id")

        outputs = model(**batch)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"📉 Epoch {epoch+1} Training Loss: {total_loss / len(train_dataloader):.4f}")

    # 🔍 Validation
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch.pop("label_id")
            outputs = model(**batch)
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f"✅ Validation Accuracy: {acc:.4f}")



from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
save_path = f"/content/drive/MyDrive/data/roberta_model_{timestamp}"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model başarıyla kaydedildi: {save_path}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import AdamW, get_cosine_schedule_with_warmup
from tqdm import tqdm
import torch.nn as nn
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Veriyi yükle ve etiketleri dönüştür
df = pd.read_csv("/content/drive/MyDrive/data/train_cleaned_augamented_cleaned.csv")
le = LabelEncoder()
df["label_id"] = le.fit_transform(df["label"])

# Oversampling uygula
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(df[["text"]], df["label_id"])
resampled_df = pd.DataFrame({"text": X_resampled["text"], "label_id": y_resampled})

# Tokenizer hazırla
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
num_labels = df["label_id"].nunique()

# Tokenization fonksiyonu
def tokenize(example):
    encoding = tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
    encoding["labels"] = int(example["label_id"])
    return encoding

# Stratified K-Fold ayarı
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Fold skorları tutmak için listeler
fold_accuracies = []
fold_losses = []
fold_precisions = []
fold_recalls = []
fold_f1s = []

# Stratified K-Fold Eğitim
for fold_idx, (train_idx, val_idx) in enumerate(skf.split(resampled_df["text"], resampled_df["label_id"])):
    print(f"\n🔵 Fold {fold_idx+1} Başlıyor...")

    # Fold'a özel train/val ayır
    train_df = resampled_df.iloc[train_idx]
    val_df = resampled_df.iloc[val_idx]

    # Fold'a özel Class Weights hesapla
    class_weights = compute_class_weight(class_weight="balanced",
                                         classes=np.unique(train_df["label_id"]),
                                         y=train_df["label_id"])
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    # Huggingface Dataset formatı
    train_dataset = Dataset.from_pandas(train_df[["text", "label_id"]])
    val_dataset = Dataset.from_pandas(val_df[["text", "label_id"]])

    tokenized_train = train_dataset.map(tokenize)
    tokenized_val = val_dataset.map(tokenize)

    tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    # DataLoader
    train_dataloader = DataLoader(tokenized_train, batch_size=16, shuffle=True)
    val_dataloader = DataLoader(tokenized_val, batch_size=16)

    # Modeli yeniden başlat
    model = RobertaForSequenceClassification.from_pretrained(
        "cardiffnlp/twitter-roberta-base-emotion",
        num_labels=num_labels,
        ignore_mismatched_sizes=True
    ).to(device)

    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    total_steps = len(train_dataloader) * 4
    scheduler = get_cosine_schedule_with_warmup(
        optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps
    )

    loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    # 📍 Early Stopping Ayarları
    best_acc = 0
    best_precision = 0
    best_recall = 0
    best_loss = float('inf')
    best_f1 = 0
    patience = 2
    patience_counter = 0

    # 📚 Eğitim
    for epoch in range(10):  # Max 10 epoch ama early stopping var
        model.train()
        total_loss = 0
        loop = tqdm(train_dataloader, desc=f"Fold {fold_idx+1} Epoch {epoch+1}")

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch.pop("labels")

            outputs = model(**batch)
            loss = loss_fn(outputs.logits, labels)
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        # 📉 Validation
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in val_dataloader:
                batch = {k: v.to(device) for k, v in batch.items()}
                labels = batch.pop("labels")
                outputs = model(**batch)
                preds = torch.argmax(outputs.logits, dim=1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        acc = accuracy_score(all_labels, all_preds)

        print(f"✅ Fold {fold_idx+1} Epoch {epoch+1} - Accuracy: {acc:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")

        # 🔥 En iyi modeli kaydet
        if f1 > best_f1:
            best_f1 = f1
            best_acc = acc
            best_precision = precision
            best_recall = recall
            best_loss = total_loss / len(train_dataloader)
            patience_counter = 0
        else:
            patience_counter += 1

        # 🔥 Early stopping kontrolü
        if patience_counter >= patience:
            print(f"⏹️ Fold {fold_idx+1} için early stopping tetiklendi! Epoch: {epoch+1}")
            break

    # 📥 Fold sonu EN İYİ SONUÇLARI kaydet
    fold_accuracies.append(best_acc)
    fold_losses.append(best_loss)
    fold_precisions.append(best_precision)
    fold_recalls.append(best_recall)
    fold_f1s.append(best_f1)

# 🎯 Eğitim Bittikten Sonra Sonuçları Yazdır
print("\n🎯 Stratified K-Fold Sonuçları:")

for i in range(len(fold_accuracies)):
    print(f"Fold {i+1} - Accuracy: {fold_accuracies[i]:.4f} | Precision: {fold_precisions[i]:.4f} | Recall: {fold_recalls[i]:.4f} | F1: {fold_f1s[i]:.4f} | Training Loss: {fold_losses[i]:.4f}")

print("\n📊 Ortalama Sonuçlar:")
print(f"Average Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Average Precision: {np.mean(fold_precisions):.4f}")
print(f"Average Recall: {np.mean(fold_recalls):.4f}")
print(f"Average F1: {np.mean(fold_f1s):.4f}")
print(f"Average Training Loss: {np.mean(fold_losses):.4f}")

# 11. Modeli Kaydet
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
save_path = f"/content/drive/MyDrive/data/roberta_model_{timestamp}"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"✅ Model başarıyla kaydedildi: {save_path}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([14, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([14]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/112000 [00:00<?, ? examples/s]

Map:   0%|          | 0/28000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1: 100%|██████████| 7000/7000 [39:32<00:00,  2.95it/s, loss=0.532]


📉 Epoch 1 Training Loss: 0.8594
✅ Validation Accuracy: 0.8062


Epoch 2:  15%|█▍        | 1042/7000 [05:52<33:35,  2.96it/s, loss=0.422]

In [ ]:
for i in range(len(fold_accuracies)):
    print(f"Fold {i+1} - Accuracy: {fold_accuracies[i]:.4f} | Precision: {fold_precisions[i]:.4f} | Recall: {fold_recalls[i]:.4f} | F1: {fold_f1s[i]:.4f} | Training Loss: {fold_losses[i]:.4f}")


In [ ]:
print("\n📊 Ortalama Sonuçlar:")
print(f"Average Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Average Precision: {np.mean(fold_precisions):.4f}")
print(f"Average Recall: {np.mean(fold_recalls):.4f}")
print(f"Average F1: {np.mean(fold_f1s):.4f}")
print(f"Average Training Loss: {np.mean(fold_losses):.4f}")
